# KTL2020 LCA
## dcm in one dirctory
## png변환 후 진행

In [1]:
import os, sys 
import pydicom as dicom
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import glob
import shutil
import tensorflow as tf
import cv2
import pandas as pd


def importOwnLib():
    if '/home/miruware/aProjects/lib' not in sys.path:
        sys.path.append('/home/miruware/aProjects/lib')
        print("lib path is successfully appended.")
    else:
        print("lib path is already exists.")

importOwnLib()
import sonyalib as sonya

import importlib
importlib.reload(sonya)


%matplotlib inline

/home/miruware/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


lib path is successfully appended.


In [ ]:
def list_files_Q(destpath):
    dcmlist = []
    qlist = []
    for file in os.listdir(destpath):
        if file.find('Q') >= 0:
            qlist.append(file)
        else:
            dcmlist.append(file)
            
    dcmlist.sort()
    qlist.sort()
    
    return dcmlist, qlist

In [36]:
PATH_BASE = '/mnt/8TBDisk/WORK/KTL2020_LCA/KTL_LCA_Original/'
PATH_PNG = '/mnt/8TBDisk/WORK/KTL2020_LCA/KTL_LCA_PNG_original/'
PATH_RESULT = '/mnt/8TBDisk/WORK/KTL2020_LCA/result20210524/'
sonya.createFolder(PATH_RESULT)
list_dcm, list_Q = list_files_Q(PATH_BASE)
list_png = os.listdir(PATH_PNG)

list_dcm, list_Q = list_files_Q(PATH_BASE)
for file_q in list_Q:
    print("=======================")
#     print(file_q)
    dcm_q = dicom.read_file(os.path.join(PATH_BASE, file_q))
    if "GraphicAnnotationSequence" in dcm_q:
        
        matching_uid = dcm_q.GraphicAnnotationSequence[nAnnotation].ReferencedImageSequence[0].ReferencedSOPInstanceUID
#         print(matching_uid)
        for file_i in [file_cases for file_cases in list_dcm if file_cases.find(file_q.split('_')[0]) >= 0]:
            with dicom.read_file(os.path.join(PATH_BASE, file_i)) as dcm_i:
                if dcm_i.SOPInstanceUID == matching_uid:
#                     print(file_i)
                    name_i = file_i
                    isMatched = True
                    list_dcm.remove(file_i)
        
        if not isMatched:
            print("matched failed")
        else:
            print("matched")
            name_png = name_i.replace('.dcm', '.jpg')
            path_target = os.path.join(PATH_PNG, name_png)
            print(name_png)
            if name_png in list_png:
                isDrawn = False
                img_ans = Image.open(path_target)
                img_black = Image.new(mode="RGB", size = img_ans.size, color=(0,0,0)) # Black 이미지

                # === Label Drawing ===
                draw_rgb = ImageDraw.Draw(img_ans)
                draw_black = ImageDraw.Draw(img_black) # draw RED label

                if "GraphicObjectSequence" in dcm_q.GraphicAnnotationSequence[nAnnotation]:
                    for nA_Object in range(len(dcm_q.GraphicAnnotationSequence[nAnnotation].GraphicObjectSequence)):
                        #print(nDcm + "\t" + dcm_q.GraphicAnnotationSequence[nAnnotation].GraphicObjectSequence[nA_Object].GraphicType)
                        obj_shape = dcm_q.GraphicAnnotationSequence[nAnnotation].GraphicObjectSequence[nA_Object].NumberOfGraphicPoints
                        obj_graphic_data = dcm_q.GraphicAnnotationSequence[nAnnotation].GraphicObjectSequence[nA_Object].GraphicData
                        if obj_shape == 5:  # Rect 
                            isDrawn = True
                            print("Object shape is Polyline")
                            polyline_data = []
                            for i in range(obj_shape):
                                polyline_data.append(tuple(obj_graphic_data[i*2:i*2+2]))
                            draw_rgb.line(polyline_data, width=label_width, fill='red', joint='curve')
                            draw_black.line(polyline_data, width=label_width, fill='red', joint='curve')
                if isDrawn:
                    print("png_saved")
                    draw_rgb_name = os.path.join(PATH_RESULT, name_png.split('.')[0] + "_RGB.png") # rgb 저장
                    draw_black_name = os.path.join(PATH_RESULT, name_png.split('.')[0] + "_black.png") # black 저장
                    shutil.copy(os.path.join(PATH_BASE, file_i), PATH_RESULT) # 원본영상 복사
                    print(draw_rgb_name)

                    img_ans.save(draw_rgb_name)
                    img_black.save(draw_black_name)
            


matched
279846_Chest_26477836_0001.jpg
5
Object shape is Polyline
5
Object shape is Polyline
png_saved
/mnt/8TBDisk/WORK/KTL2020_LCA/result20210524/279846_Chest_26477836_0001_RGB.png
matched
279847_Chest_26477838_0001.jpg
4
5
Object shape is Polyline
5
Object shape is Polyline
png_saved
/mnt/8TBDisk/WORK/KTL2020_LCA/result20210524/279847_Chest_26477838_0001_RGB.png
matched
279848_Chest_26477840_0001.jpg
2
2
5
Object shape is Polyline
png_saved
/mnt/8TBDisk/WORK/KTL2020_LCA/result20210524/279848_Chest_26477840_0001_RGB.png
matched
279849_Chest_26477842_0001.jpg
5
Object shape is Polyline
png_saved
/mnt/8TBDisk/WORK/KTL2020_LCA/result20210524/279849_Chest_26477842_0001_RGB.png
matched
279850_Chest_26477843_0002.jpg
5
Object shape is Polyline
png_saved
/mnt/8TBDisk/WORK/KTL2020_LCA/result20210524/279850_Chest_26477843_0002_RGB.png
matched
279851_Chest_26477846_0001.jpg
2
5
Object shape is Polyline
png_saved
/mnt/8TBDisk/WORK/KTL2020_LCA/result20210524/279851_Chest_26477846_0001_RGB.png
ma

## 